In [1]:
import pandas as pd
import requests as r
import json
df = pd.read_csv("./data/data.csv",sep=";")
df.shape

(200, 5)

In [2]:
def ip_to_country(ip):
    try:
        country = r.get("https://api.ipgeolocationapi.com/geolocate/{}".format(ip)).json()
#     country = "https://api.ip2country.info/ip?{}.".format(ip)
    except: 
        country = dict()
    return country

def country_currency():
    try:
        rate = r.get("https://api.exchangerate-api.com/v4/latest/EUR").json()
#     country = "https://api.ip2country.info/ip?{}.".format(ip)
    except: 
        rate  = 1
    return rate



In [8]:
df['country_json']=df["ip_address"].apply(ip_to_country)
df['rate']=df["ip_address"].apply(ip_to_country)
rate = country_currency()

In [9]:
# df["country_full_name"]=df["country_json"].apply(lambda l:l["countryCode"])
df["country_full_name"] = df["country_json"].apply(lambda l:l.get("name"))
df["country_currency"] = df["country_json"].apply(lambda l:l.get("currency_code"))
df = (df[(df["country_full_name"]!="") & (df["country_full_name"].isnull()==False)])
df["rate"] =  df["country_currency"].apply(lambda l:rate.get("rates").get(l))
df["price_clean"] = df["price"].str.extract('(\d*\.\d*)',expand=True).astype(dtype='float')
df["price_eur"] = (df["price_clean"]/df["rate"]).round(2)


/home/jorge/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/jorge/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jorge/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

In [10]:
dummy = (df["infos"].str.replace('Contains?','',case=False) 
 .str.replace('ingredients?:?','',case=False)
 .str.replace('may ','',case=False)
 .str.replace('and','',case=False)
 .str.replace(',','',case=False)
 .str.replace('\s{2,}',' ')
 .str.strip()
 .str.get_dummies(sep=' ')
)

 

In [11]:
dfinal = df.merge(dummy,right_index=True,left_index=True)

In [12]:
dfinal.columns
dfinal[["product","country_full_name","price_eur",'egg', 'fish', 'gluten', 'milk', 'mustard',
       'peanut', 'soja', 'sugar']]

,product,country_full_name,price_eur,egg,fish,gluten,milk,mustard,peanut,soja,sugar
3,"Water - Mineral, Natural",United States of America,313.85,0,0,1,0,0,0,0,0
4,Radish - Pickled,Japan,7.84,1,1,0,0,0,0,0,1
7,Dc Hikiage Hira Huba,Mexico,5.23,0,0,0,0,0,0,0,1
8,Dried Figs,Mexico,4.12,0,1,0,1,0,0,0,1
9,Pop - Club Soda Can,United States of America,771.97,0,1,0,1,0,1,0,1
10,Cinnamon Rolls,United States of America,866.17,1,1,0,0,0,1,0,1
11,Wine - Ice Wine,United States of America,609.13,0,0,0,1,1,0,0,1
12,Chicken Thigh - Bone Out,United States of America,650.64,0,0,1,0,0,0,0,1
13,Trout Rainbow Whole,China,22.62,0,0,0,0,0,1,0,0
14,Bread Crumbs - Panko,Ireland,150.89,0,0,1,1,0,0,1,1
